# Laboratorio 5: clasificación de texto usando RNN.

##### <strong>Manejo archivos y Pandas </strong>

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: [Gabriel Iturra](https://giturra.cl/)

## **Preguntas 💻** ##

El Deep Learning es la técnica que logra el estado del arte en múltiples tareas de modelamiento de tareas que utilicen datos no estructurados. En procesamiento de lenguaje natural tiene sentido utilizar un tipo de redes neuronales llamadas recurrentes debido a que toman en cuenta el orden de los elementos en una secuencia para representar los datos de entrada.

A usted se le entrega un conjunto de datos de síntomas asociados a una enfermedad y se le pide entrenar un modelo de clasificación para sugerir la enfermedad relacionada a la descripción sintomática del paciente.


Por lo que usaremos el dataset de diagnosticos en español, que hemos estado trabajando en clase.

Para realizar la clasificación del dataset usando el contenido en el archivo, se le pida crear una arquitectura basada en redes neuronales recurrentes utilizando `PyTorch`. Dado que necesitamos programar una arquitectura en redes neuronales necesitamos es necesario trabajar en Google Colab, ya que necesitaremos acceso a GPUs. Por el primer paso es cargar este archivo a Google Colab, luego realizar los siguientes pasos:


### Cargar el dataset

Para descargar el dataset debemos instalar la librería `datasets`, que contiene muchos datasets subido a `huggingface`, que ya hemos utilizados. Por lo que ejecutamos el comando:

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.2 MB/s eta 0:00:00


In [2]:
import datasets
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Con esta linea descargamos el conjunto de datos completo

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
spanish_diagnostics['train']['text']

['- ANOMALÍAS DENTOFACIALES (INCLUSO LA MALOCLUSIÓN)\n\n\n DISCREPANCIA DENTOMAXILAR',
 'OBTRUCCION FOSA NASAL DERECHA',
 'Perturbación de la actividad y de la atención Trastorno defícit atencional',
 'M7 PROLAPSO VAGINAL PARED ANTERIOR G11 G 111 ALGIA PELVICA HTA CRONICA',
 'PIEZA 3 CARIES DENTINARIA PROFUNDA PROXIMA A CAMARA PULPAR, EVALUAR POR ESPECIALIDAD',
 'pieza n 3.4 tratada endodonticamente, restaurada con ionomero y resina compuesta. Necesita protesis fija por gran pNrdida coronaria',
 'PZ. 12 TREPANADA',
 'CARCINOMA TORIODEO',
 'DISPEPSIA Y METEORISMO',
 'ASA 1 DENTICION TEMPORAL MORDIDA CRUZADA',
 '- DISMINUCION DE AGUDEZA VISUAL. - ESCATOMAS O.D.',
 'K028 OTRAS CARIES DENTALES ABFRACCIONES EN DIENTES 9, 10, 11',
 'DIENTE 1,5 TREPANADO',
 'PZA 4.6 PULPITIS IRREVERSIBLE, DOLOR A CAMBIOS TECNICOS DE LARGA DURACION, CARIES OD PROXIMA A CAMARA PULPAR.',
 'enfermedad periodontal generalizada. movilidad',
 'INSERCIÓN B AJA DEL FRENILLO LABIAL',
 'PRESENCIA DE DISPOSITIVO ANTICONC

Esto podría tardar un poco, por que se recomienda escribir el código de las siguientes secciones mientras se cargan los archivos. Una vez terminado eso. Corra la celda de pandas para asegurarse que el archivo fue cargado.

El dataset esta compuesto por dos filas, que son:

- La etiqueta que indica la categoría del diagnostico.
- El contenido del diagnostico.

Para cargar el dataset a pytorch usaremos una función similar a la del tutorial 5 para cargar crear el generador y construir el objeto del dataset en `PyTorch`.

In [11]:
def load_dataset(dataset, mode):
  return tuple(zip(dataset[mode]['label'], dataset[mode]['text']))

Luego, ejecutamos la función tanto los datasets de `train` y `test`:

In [12]:
train_dataset = load_dataset(spanish_diagnostics, 'train')
test_dataset = load_dataset(spanish_diagnostics, 'test')

Ahora el siguiente gran paso es crear Vocabulario usando `torchtext`, una sub librería de `PyTorch` que implementa diferentes clases y modulos para trabajar con métodos de Deep Learning en NLP.

### Crear vocabulario

Una vez que hemos cargado el dataset, es necesario crear nuestro vocabulario a partir de los tokens que componen nuestro dataset. Para esto, se deben seguir los siguientes pasos:

- Es necesario tokenizar el dataset.
- A partir del el dataset tokenizado, se crea el vocabulario.



In [7]:
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [13]:
tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

# Cargar el `DataLoader`

In [14]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

In [15]:
train_dataset, test_dataset  = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)


max_words = 25

Antes de cargar los dataset es necesario, definir una función que sea capaz de procesar el `batch` que se va extrayendo al entrenar este modelo. Para entender más detalles de esto, se sugiere revisar este [link](https://chadrick-kwag.medium.com/how-does-pytorch-dataloader-gather-data-from-into-batches-pytorch-f02b96bc859d).

In [16]:
def vectorize_batch(batch):

    Y, X = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X]

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y)

Con esto ya podemos crear el objeto del `DataLoader` para cargar el dataset en `PyTorch`.

In [17]:
train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)

El siguiente paso es definir el modelo de nuestra red, que se utilizará para entrenar el clasificador.

### Definir el modelo

In [18]:
from torch import nn
from torch.nn import functional as F

Un detalle más importante, es siempre definir los hyperparametros de un modelo, se sugiere utilizar los mismos del tutorial.

In [19]:
embed_len = 50
hidden_dim = 50
n_layers=1


Finalmente, usted debe agregar el código para definir el clasificador. Esto consta de dos partes:

- Primero, en el constructor `__init__` debe definir que capas son necesarias dentro del clasificador.
- Segundo, debe implementar el método `forward`, donde se ejecutan todos los pasos que usted estime conveniente en la red neuronales.

In [31]:
class RNNClassifier(nn.Module):
    def __init__(self):
        pass
        # Implementar.

    def forward(self, X_batch):
        pass
        # Implementar.

### Funciones de entrenamiento y Evaluación

Para evaluar el modelo usaremos las funciones del tutorial, no profundizaremos mucho en ellas por falta de tiempo, pero básicamente, la función `CalcValLossAndAccuracy` se encarga calcular la función de pérdida y el acuraccy en cada llamada al modelo, y la función `TrainModel`, entrena todo el modelo.

In [21]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

In [22]:


def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

### Entrenamiento del modelo.

Ahora relizamos el entrenamiento del modelo, esto podría tardar un poco, pero recuerde estar ejecutando su modelo en la GPU del google colab y no la CPU, ya que tardará muchisímo más.

In [23]:
from torch.optim import Adam

In [27]:
epochs = 10
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

100%|██████████| 69/69 [00:07<00:00,  8.63it/s]


Train Loss : 0.692
Valid Loss : 0.690
Valid Acc  : 0.524


100%|██████████| 69/69 [00:06<00:00, 10.09it/s]


Train Loss : 0.644
Valid Loss : 0.528
Valid Acc  : 0.768


100%|██████████| 69/69 [00:05<00:00, 12.11it/s]


...

Train Loss : 0.189
Valid Loss : 0.214
Valid Acc  : 0.925


100%|██████████| 69/69 [00:06<00:00,  9.88it/s]


Train Loss : 0.175
Valid Loss : 0.208
Valid Acc  : 0.933


100%|██████████| 69/69 [00:05<00:00, 12.01it/s]


Train Loss : 0.164
Valid Loss : 0.205
Valid Acc  : 0.929


### Realizar prediciones de la Red entrenada.

Para evaluar el modelo usaremos las mismas funciones del tutorial, que son:

In [28]:
def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()

Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [29]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.9291

Classification Report : 
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     15034
           1       0.92      0.94      0.93     14966

    accuracy                           0.93     30000
   macro avg       0.93      0.93      0.93     30000
weighted avg       0.93      0.93      0.93     30000


Confusion Matrix : 
[[13739  1295]
 [  832 14134]]


Ahora proponga otra arquictura en RNN, y vuelta a re-entrenar el modelo. ¿Existe una diferencia en su performance? Sí es así, ¿Porqué cree que sucede esto?

In [30]:
class RNNClassifier(nn.Module):
    def __init__(self):
      pass
    def forward(self, X_batch):
      pass